In [1]:
import os
import copy
import time
import random
from tqdm import tqdm

import PIL
from PIL import Image

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold

import torch
import torchvision
from torchvision import datasets, transforms, models

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score
# fix seeds
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 2019
seed_everything(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.enabled = False
print(device)

cuda


### KFOLD
- 5 folds for ensemble model

In [2]:
df_train = pd.read_csv('../data/train.csv')
df_train.replace(196, 0, inplace=True)
df_train['fold'] = 999

skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=False)
for index, (_, val_index) in enumerate(skf.split(df_train['img_file'], df_train['class'])):
    df_train['fold'].iloc[val_index] = index

/home/preser9191/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
def prepare_fold_dataset(df_train, fold):
    X_train = df_train[df_train['fold']==fold]['img_file']
    X_val = df_train[df_train['fold']!=fold]['img_file']
    y_train = df_train[df_train['fold']==fold]['class']
    y_val = df_train[df_train['fold']!=fold]['class']
    
    return X_train.values, X_val.values, y_train.values, y_val.values

### Prepare Dataset

In [4]:
# Batch size
batch_size = 64

# Image resize
image_size = (224, 224)

# Datapath
TRAIN_DATA_PATH = '../data/train_crop/'
TEST_DATA_PATH = '../data/test_crop/'

# torch dataset
class TrainImages(Dataset):
    def __init__(self, images, labels, mode=None, transforms=None):
        self.images = images
        self.labels = labels
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __len__(self):
        return self.images.shape[0]
        
    def __getitem__(self, idx):
        image = Image.open(TRAIN_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        label = self.labels[idx]
        
        return image, label
    
    
class TestImages(Dataset):
    def __init__(self, images, mode=None, transforms=None):
        self.images = images
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __len__(self):
        return self.images.shape[0]
        
    def __getitem__(self, idx):
        image = Image.open(TEST_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        
        return image
    
    
transform = {
    'train': transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ])
}

In [5]:
def prepare_dataloader(X_train, X_val, y_train, y_val):
    train_dataset = TrainImages(images=X_train, labels=y_train, mode='train', transforms=transform)
    val_dataset = TrainImages(images=X_val, labels=y_val, mode='val', transforms=transform)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    dataloaders = {
        'train': train_dataloader,
        'val': val_dataloader
    }

    dataset_sizes = {
        'train': len(train_dataset),
        'val': len(val_dataset)
    }
    return dataloaders, dataset_sizes

### Train

In [6]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    
    for batch_index, (batch_image, batch_target) in enumerate(dataloader):
        batch_image, batch_target = batch_image.to(device), batch_target.to(device)
        
        # forward pass
        optimizer.zero_grad()
        batch_output = model(batch_image)
        loss = criterion(batch_output, batch_target)
        
        # backward
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() / len(dataloader)
        if batch_index % (len(dataloader)//3) == 0:
            print(". . . batch: {}, batch_loss: {}".format(batch_index, loss.item()))
    return train_loss

def validate(model, dataloader, criterion, dataset_sizes):
    model.eval()
    
    val_preds = np.zeros((dataset_sizes['val'], 1))
    val_loss = 0.0

    with torch.no_grad():
        for batch_index, (batch_image, batch_target) in enumerate(dataloader):
            batch_image, batch_target = batch_image.to(device), batch_target.to(device)
            
            batch_output = model(batch_image).detach()
            _, batch_preds = torch.max(batch_output, 1)
            
            loss = criterion(batch_output, batch_target)
            val_loss += loss.item() / len(dataloader)
            
            val_preds[batch_index * batch_size: (batch_index+1) * batch_size] = batch_preds.cpu().view(-1, 1).numpy()

        val_score = f1_score(y_val, val_preds, average='micro')
        
    return val_loss, val_score

def train_model(model, dataloaders, dataset_sizes, num_epochs, fold):
    
    best_score = 0.0
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    results = {
        'best_epoch': 0,
        'best_score': 0,
    }
    
    for epoch in range(num_epochs):
        print("{} / {}".format(epoch+1, num_epochs))
        train_loss = train(model, dataloaders['train'], criterion, optimizer)
        val_loss, val_score = validate(model, dataloaders['val'], criterion, dataset_sizes)
        
        print("EPOCH: {}, train_loss: {}, val_loss: {}, val_score: {}".format(epoch+1, train_loss, val_loss, val_score))
        if val_score > best_score:
            best_score = val_score
            torch.save(model.state_dict(), './fold{}/best_model_epoch_{}_score_{}.pt'.format(fold, epoch+1, val_score))
            results['best_epoch'] = epoch+1
            results['best_score'] = best_score
            print(">>>>>>  EPOCH {}: validation score {}".format(epoch+1, val_score))
    
    return results

In [7]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)

In [8]:
# 0번째 fold
fold = 0

X_train, X_val, y_train, y_val = prepare_fold_dataset(df_train, fold)
dataloaders, dataset_sizes = prepare_dataloader(X_train, X_val, y_train, y_val)

model_res.to(device)
results = train_model(model_res, dataloaders, dataset_sizes, 20, fold)
print(results)

1 / 20
. . . batch: 0, batch_loss: 5.295028209686279
. . . batch: 11, batch_loss: 5.424709796905518
. . . batch: 22, batch_loss: 5.368289470672607
EPOCH: 1, train_loss: 5.45355698556611, val_loss: 11.21972910434969, val_score: 0.006570634318928481
>>>>>>  EPOCH 1: validation score 0.006570634318928481
2 / 20
. . . batch: 0, batch_loss: 5.306900978088379
. . . batch: 11, batch_loss: 5.251924514770508
. . . batch: 22, batch_loss: 5.335948467254639
EPOCH: 2, train_loss: 5.290252136461663, val_loss: 7.616377349822752, val_score: 0.009350518069244376
>>>>>>  EPOCH 2: validation score 0.009350518069244376
3 / 20
. . . batch: 0, batch_loss: 5.235325813293457
. . . batch: 11, batch_loss: 5.28553581237793
. . . batch: 22, batch_loss: 5.262752532958984
EPOCH: 3, train_loss: 5.270599220738264, val_loss: 5.299524603351468, val_score: 0.008971443012383118
4 / 20
. . . batch: 0, batch_loss: 5.224987983703613
. . . batch: 11, batch_loss: 5.237860679626465
. . . batch: 22, batch_loss: 5.26772594451904

In [9]:
def train_model(model, dataloaders, lr, dataset_sizes, num_epochs, fold):
    
    best_score = 0.0
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    results = {
        'best_epoch': 0,
        'best_score': 0,
    }
    
    for epoch in range(num_epochs):
        print("{} / {}".format(epoch+1, num_epochs))
        train_loss = train(model, dataloaders['train'], criterion, optimizer)
        val_loss, val_score = validate(model, dataloaders['val'], criterion, dataset_sizes)
        
        print("EPOCH: {}, train_loss: {}, val_loss: {}, val_score: {}".format(epoch+1, train_loss, val_loss, val_score))
        if val_score > best_score:
            best_score = val_score
            torch.save(model.state_dict(), './fold{}/best_model_epoch_{}_score_{}.pt'.format(fold, epoch+1, val_score))
            results['best_epoch'] = epoch+1
            results['best_score'] = best_score
            print(">>>>>>  EPOCH {}: validation score {}".format(epoch+1, val_score))
    
    return results

In [10]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/best_model_epoch_20_score_0.0731614859742229.pt'))

model_res.to(device)
results = train_model(model_res, dataloaders, 0.005, dataset_sizes, 5, fold)
print(results)

1 / 5
. . . batch: 0, batch_loss: 2.99566650390625
. . . batch: 11, batch_loss: 5.184962749481201
. . . batch: 22, batch_loss: 5.0889692306518555
EPOCH: 1, train_loss: 5.016065828727953, val_loss: 32.93907719273723, val_score: 0.026661612332575184
>>>>>>  EPOCH 1: validation score 0.026661612332575184
2 / 5
. . . batch: 0, batch_loss: 4.537700176239014
. . . batch: 11, batch_loss: 4.581735134124756
. . . batch: 22, batch_loss: 4.3808417320251465
EPOCH: 2, train_loss: 4.518917618375836, val_loss: 6.798739856289278, val_score: 0.017690169320192066
3 / 5
. . . batch: 0, batch_loss: 4.210494041442871
. . . batch: 11, batch_loss: 4.198036193847656
. . . batch: 22, batch_loss: 4.2632365226745605
EPOCH: 3, train_loss: 4.231621966217504, val_loss: 5.568101552224929, val_score: 0.03727571392469042
>>>>>>  EPOCH 3: validation score 0.03727571392469042
4 / 5
. . . batch: 0, batch_loss: 3.634830951690674
. . . batch: 11, batch_loss: 4.08501672744751
. . . batch: 22, batch_loss: 3.8468315601348877


In [11]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/best_model_epoch_5_score_0.050037907505686124.pt'))

model_res.to(device)
results = train_model(model_res, dataloaders, 0.005, dataset_sizes, 10, fold)
print(results)

1 / 10
. . . batch: 0, batch_loss: 3.2694263458251953
. . . batch: 11, batch_loss: 4.109826564788818
. . . batch: 22, batch_loss: 4.052751541137695
EPOCH: 1, train_loss: 3.9985948259180244, val_loss: 6.772104367133112, val_score: 0.05711397523376295
>>>>>>  EPOCH 1: validation score 0.05711397523376295
2 / 10
. . . batch: 0, batch_loss: 3.530360698699951
. . . batch: 11, batch_loss: 3.2438414096832275
. . . batch: 22, batch_loss: 3.507331132888794
EPOCH: 2, train_loss: 3.4877819942705575, val_loss: 10.146080228590193, val_score: 0.03866565579984837
3 / 10
. . . batch: 0, batch_loss: 3.220940589904785
. . . batch: 11, batch_loss: 3.490468740463257
. . . batch: 22, batch_loss: 3.633690357208252
EPOCH: 3, train_loss: 3.2608604864640665, val_loss: 8.143711017024133, val_score: 0.03348496335607783
4 / 10
. . . batch: 0, batch_loss: 2.8262038230895996
. . . batch: 11, batch_loss: 3.171001672744751
. . . batch: 22, batch_loss: 3.3284010887145996
EPOCH: 4, train_loss: 3.03754921392961, val_los

In [12]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)

model_res.to(device)
results = train_model(model_res, dataloaders, 0.01, dataset_sizes, 10, fold)
print(results)

1 / 10
. . . batch: 0, batch_loss: 5.308879852294922
. . . batch: 11, batch_loss: 6.237888813018799
. . . batch: 22, batch_loss: 5.424022197723389
EPOCH: 1, train_loss: 6.124887683174826, val_loss: 86282.33870967742, val_score: 0.005812484205205964
>>>>>>  EPOCH 1: validation score 0.005812484205205964
2 / 10
. . . batch: 0, batch_loss: 5.3067240715026855
. . . batch: 11, batch_loss: 5.308599472045898
. . . batch: 22, batch_loss: 5.280768871307373
EPOCH: 2, train_loss: 5.2822365471811, val_loss: 16.54715223850743, val_score: 0.008213292898660601
>>>>>>  EPOCH 2: validation score 0.008213292898660601
3 / 10
. . . batch: 0, batch_loss: 5.270358085632324
. . . batch: 11, batch_loss: 5.284801483154297
. . . batch: 22, batch_loss: 5.294198989868164
EPOCH: 3, train_loss: 5.2784308664726485, val_loss: 5.311260657925761, val_score: 0.007707859489512257
4 / 10
. . . batch: 0, batch_loss: 5.293835163116455
. . . batch: 11, batch_loss: 5.257975101470947
. . . batch: 22, batch_loss: 5.257778167724

KeyboardInterrupt: 